# 5章 顧客の退会を予測する１０本ノック

引き続き、スポーツジムの会員データを使って顧客の行動を分析していきます。  
３章では顧客の全体像を把握し、4章では数ヶ月利用している顧客の来月の利用回数の予測を行いました。   
ここでは、教師あり学習の分類を用いて、顧客の退会予測を取り扱います。

### ノック41：データを読み込んで利用データを整形しよう

In [1]:
!ls

'5章 顧客の退会を予測する１０本ノック.ipynb'	      customer_join.csv
'5章 顧客の退会を予測する１０本ノック_answer.ipynb'   use_log_months.csv


In [76]:
import pandas as pd
df_customer_join = pd.read_csv("./customer_join.csv")
df_use_log_months = pd.read_csv("./use_log_months.csv")


In [77]:
df_use_log_months.head()

,年月,customer_id,count
0,201804,AS002855,4
1,201804,AS009013,2
2,201804,AS009373,3
3,201804,AS015315,6
4,201804,AS015739,7


In [84]:
# 当月と一ヶ月前の利用履歴のみのデータを作成
df_41_uselog = df_use_log_months.copy()

df_41_uselog["年月"] = pd.to_datetime(df_41_uselog["年月"], format="%Y%m")
df_41_uselog.set_index("年月", inplace=True)

g = df_41_uselog.groupby("customer_id")
tmp = list()
for customer, df in g:
    df["count_1"]= df["count"].shift()
    tmp.append(df.dropna())
uselog = pd.concat(tmp)


/home/otowa/miniconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [85]:
uselog.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 32650 entries, 2018-05-01 to 2019-03-01
Data columns (total 3 columns):
customer_id    32650 non-null object
count          32650 non-null int64
count_1        32650 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 1020.3+ KB


In [71]:
df_42_customer.head()

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,prev_month_of_end_date
708,TS511179,XXXXXX,C01,F,2016-05-01,2018-04-30,CA1,1,オールタイム,10500,通常,3.0,3.0,3,3,0,2018-04-30,23,2018-03-30
729,TS443736,XXXX,C02,M,2016-05-01,2018-04-30,CA1,1,デイタイム,7500,通常,3.0,3.0,3,3,0,2018-04-30,23,2018-03-30
730,HD542886,XX,C01,M,2016-05-01,2018-04-30,CA1,1,オールタイム,10500,通常,1.0,1.0,1,1,0,2018-04-30,23,2018-03-30
770,HD597545,XXXXX,C03,F,2016-06-01,2018-05-31,CA1,1,ナイト,6000,通常,3.5,3.5,4,3,1,2018-05-31,23,2018-04-30
785,HI749296,XXXXX,C01,M,2016-06-01,2018-05-31,CA1,1,オールタイム,10500,通常,3.0,3.0,3,3,0,2018-05-31,23,2018-04-30


In [72]:
uselog[uselog["customer_id"] == "TS511179"]

,customer_id,count,count_1
年月,,,


### ノック42：退会前月の退会顧客データを作成しよう

In [93]:
# 退会した顧客データだけ取得
df_42_customer = df_customer_join[df_customer_join["is_deleted"]==1].copy()

# 退会前月
from dateutil.relativedelta import relativedelta
df_42_customer["end_date"] = pd.to_datetime(df_42_customer["end_date"])
df_42_customer["prev_month_of_end_date"] = df_42_customer["end_date"].apply(lambda x: x-relativedelta(months=1))
#df_42_customer[["end_date", "prev_month_of_end_date"]].head()

# uselog とマージ
x = df_42_customer.merge(uselog.reset_index(), on="customer_id")
x = x.sort_values(by=["customer_id", "年月"] )
x = x.groupby("customer_id").last()
#x[x["customer_id"] == "AS002855"]
#x[["customer_id", "年月"] ]
x.head().T

customer_id,AS008805,AS015746,AS019120,AS025956,AS029624
name,XXXXX,XXXXX,XXXX,XXXXXX,XXXXX
class,C01,C01,C01,C01,C01
gender,M,M,M,M,M
start_date,2018-06-07,2017-12-01,2018-08-07,2018-05-09,2018-05-08
end_date,2019-03-31 00:00:00,2018-08-31 00:00:00,2018-10-31 00:00:00,2019-02-28 00:00:00,2019-01-31 00:00:00
campaign_id,CA2,CA3,CA1,CA2,CA2
is_deleted,1,1,1,1,1
class_name,オールタイム,オールタイム,オールタイム,オールタイム,オールタイム
price,10500,10500,10500,10500,10500
campaign_name,入会費半額,入会費無料,通常,入会費半額,入会費半額


### ノック43：継続顧客のデータを作成しよう

### ノック44：予測する月の在籍期間を作成しよう

### ノック45：欠損値を除去しよう

### ノック46：文字列型の変数を処理できるように整形しよう

### ノック47：決定木を用いて退会予測モデルを作成してみよう

### ノック48：予測モデルの評価を行ない、モデルのチューニングをしてみよう

### ノック49：モデルに寄与している変数を確認しよう

### ノック50：顧客の退会を予測しよう